In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
spark = SparkSession.builder.getOrCreate()

22/06/19 13:09:26 WARN Utils: Your hostname, sam-pc resolves to a loopback address: 127.0.1.1; using 10.175.240.170 instead (on interface wlp2s0)
22/06/19 13:09:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/19 13:09:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Create the schema of the streamed files (check the column names and types from the CSV files)

In [ ]:
df = spark.read.csv('kospi/KOSPI_STOCK_0.csv', header=True, inferSchema=True)

In [ ]:
df = df.withColumnRenamed('_c0', 'ID')

In [ ]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [ ]:
schema = df.schema

### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [ ]:
reader = spark.readStream.format('csv').schema(schema).load('kospi/')

### Make sure the sataframe is streaming the files from the folder

In [ ]:
reader.isStreaming

True

### Create a stream writer into memory and specify the query name "stock:

In [ ]:
writer = reader.writeStream.outputMode('append').format('memory').queryName('stock')

### Start the write stream and make sure it works (read all columns from the table)

In [ ]:
query = writer.start(queryName='stock')

22/06/19 13:24:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ef9749c7-fa0b-4a35-a6b7-cc43652bc190. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/19 13:24:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
all_data = spark.sql('select * from stock')

In [ ]:
all_data.show(100)

+----+----------+------------+------------+------------+------------+------------+------+
|  ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+----+----------+------------+------------+------------+------------+------------+------+
|null|      Date|        null|        null|        null|        null|        null|  null|
| 240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|
| 241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|
| 242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656|
| 243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|
| 244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|
| 245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|
| 246|2000-12-13|27651.599609|     29286.5|27651.599609|28469.099609|26903.234375|270385|
| 247|2000

### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [ ]:
NoNull_df = all_data.dropna(how='any')

In [ ]:
diff = round(NoNull_df['high'] - NoNull_df['low'], 5)
clean_df = NoNull_df.withColumn('diff', diff)

In [ ]:
clean_df.show(5)

+---+----------+------------+------------+------------+------------+------------+------+----------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|      diff|
+---+----------+------------+------------+------------+------------+------------+------+----------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019| 995.20117|
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791| 710.80078|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656| 533.10156|
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|1066.29883|
|244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|1208.40039|
+---+----------+------------+------------+------------+------------+------------+------+----------+
only showing top 5 rows



#### SAVE

In [ ]:
# save 
clean_df.write.format('csv').option('header','true').save('output/myData1', mode='overwrite')

In [ ]:
query.stop()

---
### Create a new write stream using the new generated dataframe and call the generate table "modified_data"

In [ ]:
reader1 = spark.readStream.format('csv').\
                            option('header','true').\
                                schema(clean_df.schema).\
                                    load('output/myData1/')

In [ ]:
writer1 = reader1.writeStream.outputMode('append').\
                                format('memory').\
                                    queryName('modified_data')

In [ ]:
query = writer1.start(queryName='modified_data')

22/06/19 13:49:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a54c6eac-6229-4b71-bf76-00ab938546be. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/19 13:49:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
all_data = spark.sql('select * from modified_data')

In [ ]:
all_data = all_data.withColumnRenamed('Adj Close', 'Adj_Close')
all_data.show()

+---+----------+------------+------------+------------+------------+------------+------+----------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj_Close|Volume|      diff|
+---+----------+------------+------------+------------+------------+------------+------+----------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019| 995.20117|
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791| 710.80078|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656| 533.10156|
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|1066.29883|
|244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|1208.40039|
|245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560| 639.79883|
|246|2000-12-13|27651.599609|     29286.5|27651.599609|28469.099609|26903.234375|270385|1634.90039|


### Write the generated data into files instead of the memory. 

In [ ]:
# save as parquet
all_data.write.format('parquet').option('header','true').save('output/myData2', mode='overwrite')

### Stop the query. Now, try reading the generated parquet files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [ ]:
query.stop()

In [ ]:
finalDF = spark.read.format('parquet').\
                            option('header','true').\
                                schema(all_data.schema).\
                                    load('output/myData2/')

In [ ]:
finalDF.show()

+---+----------+------------+------------+------------+------------+------------+------+----------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj_Close|Volume|      diff|
+---+----------+------------+------------+------------+------------+------------+------+----------+
|140|2000-07-18|     25021.5|25732.300781|24879.300781|24950.400391|23578.070313| 48253|     853.0|
|141|2000-07-19|     25021.5|25732.300781|24879.300781|24950.400391|23578.070313| 48253|     853.0|
|142|2000-07-20|24523.900391|26158.800781|23813.099609|24666.099609|23309.408203| 79062|2345.70117|
|143|2000-07-21|24523.900391|25234.699219|24523.900391|24950.400391|23578.070313| 35029| 710.79883|
|144|2000-07-24|24950.400391|     25448.0|24559.400391|24950.400391|23578.070313| 42626| 888.59961|
|145|2000-07-25|25519.099609|26016.699219|     24595.0|25625.699219|24216.228516|142789|1421.69922|
|146|2000-07-26|25590.199219|28220.300781|25590.199219|27367.300781|25862.035156|208064|2630.10156|


### Sort the dataframe based on the ID

In [ ]:
finalDFSorted = finalDF.sort('ID')
finalDFSorted.show()

+---+----------+------------+------------+------------+------------+------------+------+----------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj_Close|Volume|      diff|
+---+----------+------------+------------+------------+------------+------------+------+----------+
|  0|2000-01-04|22817.900391|25696.800781|22817.900391|24879.300781|23510.880859|108745|2878.90039|
|  1|2000-01-05|24523.900391|26229.900391|23670.900391|24417.300781|23074.294922|175990|    2559.0|
|  2|2000-01-06|24381.699219|24666.099609|22746.800781|22817.900391|21562.865234| 71746|1919.29883|
|  3|2000-01-07|     22036.0|24879.300781|     22036.0|23884.199219|22570.513672|120984|2843.30078|
|  4|2000-01-10|24879.300781|25519.099609|23813.099609|24061.900391|22738.439453|151371|    1706.0|
|  5|2000-01-11|     24168.5|     25021.5|23955.199219|24239.599609|22906.365234| 95943|1066.30078|
|  6|2000-01-12|     24168.5|24452.800781|23457.599609|23670.900391|22368.947266| 61899| 995.20117|
